<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2025notebooks/2025_1121StableBaselines3_1_getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Baselines3 チュートリアル - はじめに<!-- # Stable Baselines3 Tutorial - Getting Started -->

Github repo: https://github.com/araffin/rl-tutorial-jnrr19/tree/sb3/

Stable-Baselines3: https://github.com/DLR-RM/stable-baselines3

Documentation: https://stable-baselines3.readthedocs.io/en/master/

SB3-Contrib: https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

RL Baselines3 zoo: https://github.com/DLR-RM/rl-baselines3-zoo

[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) は Stable Baselines3 を用いた強化学習（RL）の訓練フレームワークである。
<!-- [RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) is a training framework for Reinforcement Learning (RL), using Stable Baselines3. -->

訓練、エージェントの評価、ハイパーパラメータの調整、結果のプロット、動画の記録のためのスクリプトを提供。
<!-- It provides scripts for training, evaluating agents, tuning hyperparameters, plotting results and recording videos. -->


## はじめに<!-- ## Introduction-->

このノートブックでは、stable baselinesライブラリの基本的な使い方を学ぶ。具体的には、強化学習モデルの作成方法、学習方法、評価方法だ。全てのアルゴリズムが同じインターフェースを共有しているため、アルゴリズムを切り替えるのがいかに簡単かを見ていく。
<!--In this notebook, you will learn the basics for using stable baselines library: how to create a RL model, train it and evaluate it. Because all algorithms share the same interface, we will see how simple it is to switch from one algorithm to another. -->


## Pipを使用した依存関係とStable Baselines3のインストール<!-- ## Install Dependencies and Stable Baselines3 Using Pip-->

完全な依存関係の一覧は[README](https://github.com/DLR-RM/stable-baselines3)に記載されている。
<!--List of full dependencies can be found in the [README](https://github.com/DLR-RM/stable-baselines3). -->


```
pip install stable-baselines3[extra]
```

In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"

## 輸入 Imports

Stable-Baselines3 は [gymインターフェース](https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html) に準拠した環境で動作する。
利用可能な環境の一覧は [こちら](https://gymnasium.farama.org/environments/classic_control/) で確認できる。
全てのアルゴリズムが全ての行動空間で動作するわけではない。詳細は [まとめ表](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html) を参照。

<!-- Stable-Baselines3 works on environments that follow the [gym interface](https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html).
You can find a list of available environment [here](https://gymnasium.farama.org/environments/classic_control/).

Not all algorithms can work with all action spaces, you can find more in this [recap table](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html) -->

In [2]:
import gymnasium as gym
import numpy as np

最初にインポートする必要があるのは RL モデルである。
どの問題に何が使えるかを知るには、ドキュメントを確認。
<!-- The first thing you need to import is the RL model, check the documentation to know what you can use on which problem -->

In [ ]:
from stable_baselines3 import PPO

次にインポートする必要があるのは、ネットワーク（ポリシー関数／価値関数用）を作成するために使用するポリシークラスだ。
このステップは任意だ。コンストラクタで直接文字列を使用できるからだ：
<!-- The next thing you need to import is the policy class that will be used to create the networks (for the policy/value functions).
This step is optional as you can directly use strings in the constructor:-->

`PPO(MlpPolicy, env)` の代わりに `PPO('MlpPolicy', env) `

なお、`SAC`のような一部のアルゴリズムは独自の `MlpPolicy` を持つ。そのため、ポリシーには文字列を使用するのが推奨される。

<!--```PPO('MlpPolicy', env)``` instead of ```PPO(MlpPolicy, env)```

Note that some algorithms like `SAC` have their own `MlpPolicy`, that's why using string for the policy is the recommended option. -->

In [4]:
from stable_baselines3.ppo.policies import MlpPolicy

## Gym 環境を作成し、エージェントを実体化<!-- ## Create the Gym env and instantiate the agent-->

本例では、古典的な制御問題である CartPole 環境を使用する。
<!-- For this example, we will use CartPole environment, a classic control problem. -->

「ポールは非作動関節でカートに取り付けられており、カートは摩擦のない軌道上を移動する。システムはカートに +1 または -1 の力を加えることで制御される。振り子は垂直に立っており、倒れないようにすることが目標である。ポールが垂直に保たれた各時間ステップごとに +1 の報酬が与えられる。」
<!--"A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. " -->

Cartpole 環境: [https://gymnasium.farama.org/environments/classic_control/cart_pole/](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

<center>

![Cartpole](https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif)
</center>

MlpPolicy を選んだのは、CartPole 課題の観測が画像ではなく特徴ベクトルだからである。
<!-- We chose the MlpPolicy because the observation of the CartPole task is a feature vector, not images.-->

使用する行為の種類（離散/連続）は、環境の行為空間から自動的に推測される。
<!-- The type of action to use (discrete/continuous) will be automatically deduced from the environment action space -->


ここでは [近似方針最適化](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html) アルゴリズムを使用している。これはアクター・クリティック手法であり、価値関数を用いて方針勾配降下法を改善する（分散を低減する）。
<!-- Here we are using the [Proximal Policy Optimization](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html) algorithm, which is an Actor-Critic method: it uses a value function to improve the policy gradient descent (by reducing the variance). -->

これは [A2C](https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html)（複数のワーカを持ち、探索にエントロピーボーナスを用いる）と [TRPO](https://stable-baselines.readthedocs.io/en/master/modules/trpo.html) のアイデアを組み合わせたものだ （信頼領域を用いて安定性を向上させ、性能の急激な低下を回避する）のアイデアを組み合わせている。
<!-- It combines ideas from [A2C](https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html) (having multiple workers and using an entropy bonus for exploration) and [TRPO](https://stable-baselines.readthedocs.io/en/master/modules/trpo.html) (it uses a trust region to improve stability and avoid catastrophic drops in performance). -->


PPO はオンポリシーアルゴリズムであり、ネットワークを更新するために使用する軌道は最新の方針を用いて収集されなければならない。
<!-- PPO is an on-policy algorithm, which means that the trajectories used to update the networks must be collected using the latest policy. -->

通常、[DQN](https://stable-baselines.readthedocs.io/en/master/modules/dqn.html)、 [SAC](https://stable-baselines3.readthedocs.io/en/master/modules/sac.html)や[TD3](https://stable-baselines3.readthedocs.io/en/master/modules/td3.html)といったオフポリシーアルゴリズムよりサンプル効率は劣るが、実時間でははるかに高速である。
<!--It is usually less sample efficient than off-policy alorithms like [DQN](https://stable-baselines.readthedocs.io/en/master/modules/dqn.html), [SAC](https://stable-baselines3.readthedocs.io/en/master/modules/sac.html) or [TD3](https://stable-baselines3.readthedocs.io/en/master/modules/td3.html), but is much faster regarding wall-clock time. -->


In [ ]:
env = gym.make("CartPole-v1")

model = PPO(MlpPolicy, env, verbose=0)

エージェントを評価するための補助関数を作成：
<!-- We create a helper function to evaluate the agent: -->

In [6]:
from stable_baselines3.common.base_class import BaseAlgorithm


def evaluate(
    model: BaseAlgorithm,
    num_episodes: int = 100,
    deterministic: bool = True,
) -> float:
    """
    Evaluate an RL agent for `num_episodes`.

    :param model: the RL Agent
    :param env: the gym Environment
    :param num_episodes: number of episodes to evaluate it
    :param deterministic: Whether to use deterministic or stochastic actions
    :return: Mean reward for the last `num_episodes`
    """
    # This function will only work for a single environment
    vec_env = model.get_env()
    obs = vec_env.reset()
    all_episode_rewards = []
    for _ in range(num_episodes):
        episode_rewards = []
        done = False
        # Note: SB3 VecEnv resets automatically:
        # https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecenv-api-vs-gym-api
        # obs = vec_env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            # `deterministic` is to use deterministic actions
            action, _states = model.predict(obs, deterministic=deterministic)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, _info = vec_env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print(f"Mean reward: {mean_episode_reward:.2f} - Num episodes: {num_episodes}")

    return mean_episode_reward

未訓練のエージェントを評価。これはランダムなエージェントであるべきである。
<!-- Let's evaluate the un-trained agent, this should be a random agent. -->

In [ ]:
# Random Agent, before training
mean_reward_before_train = evaluate(model, num_episodes=100, deterministic=True)

Stable-Baselines は既にその補助機能を提供している：
<!-- Stable-Baselines already provides you with that helper: -->

In [8]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, warn=False)

print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")

## エージェントの訓練と評価
<!-- ## Train the agent and evaluate it -->

In [ ]:
# Train the agent for 10000 steps
model.learn(total_timesteps=10_000)

In [ ]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

訓練はうまくいったらしく、報酬の平均値がかなり上昇した
<!-- Apparently the training went well, the mean reward increased a lot ! -->

### 動画記録の準備<!-- ### Prepare video recording -->

In [12]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [13]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay


def show_videos(video_path="", prefix=""):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

[VecVideoRecorder](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder) ラッパーを使って動画を録画する。このラッパーについては次のノートブックで学ぶ。
<!-- We will record a video using the [VecVideoRecorder](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder) wrapper, you will learn about those wrapper in the next notebook. -->

In [14]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv


def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

### 訓練済みエージェントの可視化<!-- ### Visualize trained agent -->


In [ ]:
record_video("CartPole-v1", model, video_length=500, prefix="ppo-cartpole")

In [ ]:
show_videos("videos", prefix="ppo")

## ボーナス：1 行で RL モデルを訓練<!-- ## Bonus: Train a RL Model in One Line-->

使用するポリシークラスは推論され、環境は自動生成される。これは両方が[登録済み](https://stable-baselines3.readthedocs.io/en/master/guide/quickstart.html)だから機能する。
<!--The policy class to use will be inferred and the environment will be automatically created. This works because both are [registered](https://stable-baselines3.readthedocs.io/en/master/guide/quickstart.html). -->

In [ ]:
model = PPO('MlpPolicy', "CartPole-v1", verbose=1).learn(1000)

## 結論<!-- ## Conclusion-->

このノートブックでは次のことを学んだ：
- stable baselines3 を使って RL モデルを定義し訓練する方法。たった一行のコードで済む
<!--In this notebook we have seen:
- how to define and train a RL model using stable baselines3, it takes only one line of code ;) -->